# Connect with `keep_artifacts_local=True`

In [ ]:
# this shouldn't run in a notebook, but in the unit tests
# currently still limited because of inability to load multiple instances

import pytest
import lamindb_setup as ln_setup
from pathlib import Path

ln_setup.login("testuser1")
ln_setup.init(storage="s3://lamindb-ci/keep-artifacts-local-setup")

assert ln_setup.settings.instance.name == "keep-artifacts-local-setup"
assert ln_setup.settings.instance.storage.type_is_cloud
assert (
    ln_setup.settings.instance.storage.root_as_str
    == "s3://lamindb-ci/keep-artifacts-local-setup"
)
assert (
    ln_setup.settings.instance._sqlite_file.as_posix()
    == f"s3://lamindb-ci/keep-artifacts-local-setup/{ln_setup.settings.instance._id.hex}.lndb"  # noqa
)

In [ ]:
with pytest.raises(ValueError) as error:
    ln_setup.settings.instance.storage_local
assert error.exconly() == "ValueError: `keep_artifacts_local` is not enabled for this instance."

In [ ]:
ln_setup.settings.instance._keep_artifacts_local = True
with pytest.raises(ValueError) as error:
    ln_setup.settings.instance.storage_local

In [ ]:
# now set local storage location
ln_setup.settings.instance.storage_local = "./my_storage_local"

assert (
    ln_setup.settings.instance.storage_local.root.as_posix()
    == Path("./my_storage_local").resolve().as_posix()
)
assert (ln_setup.settings.instance.storage_local.root / ".lamindb/_is_initialized").read_text() == ln_setup.settings.instance.storage_local.uid
assert ln_setup.settings.instance.storage_local is not None
# the remote storage location is still in the regular slot
assert ln_setup.settings.instance.storage.root.as_posix() == "s3://lamindb-ci/keep-artifacts-local-setup"

Another one:

In [ ]:
ln_setup.settings.instance.storage_local = "./my_storage_local2"

In [ ]:
assert (
    ln_setup.settings.instance.storage_local.root.as_posix()
    == Path("./my_storage_local2").resolve().as_posix()
)
assert (ln_setup.settings.instance.storage_local.root / ".lamindb/_is_initialized").read_text() == ln_setup.settings.instance.storage_local.uid

See whether we can repeat this:

In [ ]:
ln_setup.settings.instance.storage_local = "./my_storage_local2"

And back to the initial one:

In [ ]:
ln_setup.settings.instance.storage_local = "./my_storage_local"

Add a test file:

In [ ]:
test_file = (ln_setup.settings.instance.storage_local.root / ".lamindb/test_file.txt")
test_file.write_text("test")

In [ ]:
ln_setup.settings.instance.storage_local.root.view_tree()

In [ ]:
with pytest.raises(ln_setup.core.upath.InstanceNotEmpty):
    ln_setup.delete("keep-artifacts-local-setup", force=True)

In [ ]:
test_file.unlink()

In [ ]:
ln_setup.delete("keep-artifacts-local-setup", force=True)